In [ ]:
import pandas as pd
import os
import scanpy as sc
import anndata as ad
human_datasets = [] # list of human sample IDs

In [ ]:
# remove intra-dataset redundency, then obtain robust programs
path_to_all_h5ad_files = ""
path_to_weights_file = ""
path_to_file = ""
whole = list(os.walk(path_to_all_h5ad_files))[0][2]

nonreduncency_threshold = 10
robust_threshold = 10   # As we used the seed 'nndsvd' for better NMF performance, there is no need to remove repetitive programs from different k.

nonredundent_programs = pd.DataFrame(range(50),columns=["program_template"]) # Initiate the container
nonredundent_robust_programs_count_per_sample = []

used_samples = []
for id in whole:
    if id[:10] in human_datasets:
        used_samples.append(id)

for id in used_samples:
    nonredundent_programs_list = []     # Initiate the container for the first sample
    if id[:10] in human_datasets:
        temp = pd.read_csv(path_to_weights_file+id[:10]+"_weights.csv")
        temp.index = temp["Unnamed: 0"]
        del temp["Unnamed: 0"]    # Get weights from the first sample
        for i in temp.columns:
            program_temp = temp[i].sort_values(ascending=False).index[:50].to_list()    # Get a 50 top-weighted gene column
            if nonredundent_programs_list==[]:
                nonredundent_programs_list.append(program_temp)     # Save the first column
            else:
                if_nonredundency_any = []
                for j in nonredundent_programs_list:
                    coexisting_genes = set(j).intersection(set(program_temp))       # Compare non-first columns to former columns from the same sample, and remove redundent programs
                    if len(coexisting_genes)>=nonreduncency_threshold:
                        if_nonredundency_any.append(False)
                if all(if_nonredundency_any):
                    nonredundent_programs_list.append(program_temp)
                    nonredundent_programs[id[:10]+"_rank4_9_nruns10.RDS.30."+str(temp.columns.to_list().index(i)+1)] = program_temp # Save non-redundent programs
                    try:
                        nonredundent_robust_programs_count_per_sample[used_samples.index(id)] += 1
                    except:
                        nonredundent_robust_programs_count_per_sample.append(1)
                else:
                    continue
del nonredundent_programs["program_template"]       # Remove the initializing column after the first sample processed

robust_programs = pd.DataFrame(range(50),columns=["program_template"])      # Initiate the container for all samples
nonredundency_program_from = [nonredundent_programs.columns[i].split("_")[0] for i in range(len(nonredundent_programs.columns))]
for id in nonredundent_programs.columns:        # Compare with programs from other samples
    program_temp = nonredundent_programs[id].to_list()
    if_any_robust = []      # Any 'True' element indicates it is robust
    for i in nonredundency_program_from:
        if i != id.split("_")[0]:
            program_from_other = nonredundent_programs[nonredundent_programs.columns[nonredundency_program_from.index(i)]].to_list()
            coexisting_genes = set(program_from_other).intersection(set(program_temp))
            if len(coexisting_genes) >= robust_threshold:
                if_any_robust.append(True)
    if any(if_any_robust):
        robust_programs[id] = program_temp
    else:
        continue
del robust_programs["program_template"]


robust_programs.to_csv(path_to_file+"human_robust_programs.csv", header=True, index=False)